In [4]:
import numpy as np
import pandas as pd
import sklearn as sk
import pandas as pd
import seaborn as sns
import keras
from keras import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score



d = pd.read_csv('houses_edited.csv')


features = ["bedrooms", "bathrooms", "sqft", "parking", 
            "mean_district_income", "Plex", "Semi-Detached", 
            "Store W/Apt/Offc", "Co-Op Apt", "Co-Ownership Apt", 
            "Link", "Comm Element Condo", "Att/Row/Twnhouse", 
            "Condo Townhouse", "Detached", "Condo Apt"] #define list of features to input in nerual network


df = pd.DataFrame(data = d)


def remove_beds(x): #function to remove the word "beds" from column of number of beds
    x = x.replace(" beds", "")
    return x
def calc_beds(j): #function to calculate number of beds from string
    answer = sum(int(i) for i in j.split('+'))
    return answer


df.bedrooms = df.bedrooms.apply(remove_beds) 
df.bedrooms = df.bedrooms.apply(calc_beds)

In [5]:
df = df.drop(["description", "mls", "bedrooms_ag", "bedrooms_bg", "full_link", "full_address", "title",
              "final_price_transformed", "final_price_log", "city_district", "district_code",
              "index", "lat", "long", "list_price"], axis = 1) #Remove insignificant values

#Fill missing sqft values based on type of property

sqft_Store = 0
sqft_Link = 0
sqft_Twn = 0
sqft_Plex = 0
sqft_Detached = 0
sqft_Semi_detached = 0
sqft_Store_count = 0
sqft_Link_count = 0
sqft_Twn_count = 0
sqft_Plex_count = 0
sqft_Detached_count = 0
sqft_Semi_detached_count = 0


for index, row in df.iterrows():
    if pd.notnull(row["sqft"]) and row["type"] == "Store W/Apt/Offc":
      sqft_Store += row["sqft"]
      sqft_Store_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Link":
      sqft_Link += row["sqft"]
      sqft_Link_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Att/Row/Twnhouse":
      sqft_Twn += row["sqft"]
      sqft_Twn_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Plex":
      sqft_Plex += row["sqft"]
      sqft_Plex_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Detached":
      sqft_Detached += row["sqft"]
      sqft_Detached_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Semi-Detached":
      sqft_Semi_detached += row["sqft"]
      sqft_Semi_detached_count += 1

avg_Store = sqft_Store/sqft_Store_count
avg_Link = sqft_Link/sqft_Link_count
avg_Twn = sqft_Twn/sqft_Twn_count
avg_Plex = sqft_Plex/sqft_Plex_count
avg_Detached = sqft_Detached/sqft_Detached_count
avg_Semi_detached = sqft_Semi_detached/sqft_Semi_detached_count


for index, row in df.iterrows():
  #print(row["type"], row["sqft"])
  if pd.isnull(row["sqft"]) and row["type"] == "Store W/Apt/Offc":
    df.at[index, "sqft"] = avg_Store

  if pd.isnull(row["sqft"]) and row["type"] == "Link":
    df.at[index, "sqft"] = avg_Link

  if pd.isnull(row["sqft"]) and row["type"] == "Att/Row/Twnhouse":
    df.at[index, "sqft"] = avg_Twn

  if pd.isnull(row["sqft"]) and row["type"] == "Plex":
    df.at[index, "sqft"] = avg_Plex

  if pd.isnull(row["sqft"]) and row["type"] == "Detached":
    df.at[index, "sqft"] = avg_Detached

  if pd.isnull(row["sqft"]) and row["type"] == "Semi-Detached":
    df.at[index, "sqft"] = avg_Semi_detached


dummy_type = pd.get_dummies(df["type"]) #allow 
df = pd.concat([df, dummy_type], axis = 1)
df = df.drop(["type"], axis = 1)

In [31]:
train, test = train_test_split(df, test_size = 0.20, random_state = 0) #Split data into 2 parts, 
                                                                       #training and testing data, with ratio 8:2 

x_train = train[features]
y_train = train[['final_price']]

model = Sequential()

model.add(Dense(64, input_dim = 16,activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))

opt = keras.optimizers.Adam(learning_rate = 0.03) 

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['MeanAbsolutePercentageError'])
model.fit(x_train, y_train, epochs = 1500, batch_size = 200, shuffle = False, verbose = 2)
#training = model.predict(x_train)
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy: %.2f' % (accuracy))

predictions = model.predict(x_train)
#print(predictions)
for i in range(10):
  print("%d, expected: %d" % (predictions[i], y_train.iloc[i]))

# Predicting prices from test set
# x_test = test[features]
# y_test = test[['final_price']]


# print(model.evaluate(x_test, y_test))


#Best hyperparameter result
# MSPE: 17.95%
# Neural layers: 64, 32, 1
# Learning rate: 0.03
# Epochs: 1500
# Batch size; 200


Epoch 1/1500
61/61 - 0s - loss: 374811885568.0000 - mean_absolute_percentage_error: 43.2305
Epoch 2/1500
61/61 - 0s - loss: 215613292544.0000 - mean_absolute_percentage_error: 30.1086
Epoch 3/1500
61/61 - 0s - loss: 175385706496.0000 - mean_absolute_percentage_error: 28.4178
Epoch 4/1500
61/61 - 0s - loss: 176308600832.0000 - mean_absolute_percentage_error: 28.3701
Epoch 5/1500
61/61 - 0s - loss: 176165617664.0000 - mean_absolute_percentage_error: 28.3590
Epoch 6/1500
61/61 - 0s - loss: 176306012160.0000 - mean_absolute_percentage_error: 28.3609
Epoch 7/1500
61/61 - 0s - loss: 176415473664.0000 - mean_absolute_percentage_error: 28.3540
Epoch 8/1500
61/61 - 0s - loss: 176474734592.0000 - mean_absolute_percentage_error: 28.3404
Epoch 9/1500
61/61 - 0s - loss: 176477421568.0000 - mean_absolute_percentage_error: 28.3221
Epoch 10/1500
61/61 - 0s - loss: 176434184192.0000 - mean_absolute_percentage_error: 28.3013
Epoch 11/1500
61/61 - 0s - loss: 176362881024.0000 - mean_absolute_percentage_e

Epoch 90/1500
61/61 - 0s - loss: 156792651776.0000 - mean_absolute_percentage_error: 28.2089
Epoch 91/1500
61/61 - 0s - loss: 156423143424.0000 - mean_absolute_percentage_error: 28.1755
Epoch 92/1500
61/61 - 0s - loss: 156388835328.0000 - mean_absolute_percentage_error: 28.1993
Epoch 93/1500
61/61 - 0s - loss: 156191571968.0000 - mean_absolute_percentage_error: 28.3060
Epoch 94/1500
61/61 - 0s - loss: 156314271744.0000 - mean_absolute_percentage_error: 28.1744
Epoch 95/1500
61/61 - 0s - loss: 156328312832.0000 - mean_absolute_percentage_error: 28.1759
Epoch 96/1500
61/61 - 0s - loss: 156670443520.0000 - mean_absolute_percentage_error: 28.1970
Epoch 97/1500
61/61 - 0s - loss: 156380250112.0000 - mean_absolute_percentage_error: 28.0799
Epoch 98/1500
61/61 - 0s - loss: 156061188096.0000 - mean_absolute_percentage_error: 28.3184
Epoch 99/1500
61/61 - 0s - loss: 156316844032.0000 - mean_absolute_percentage_error: 28.1953
Epoch 100/1500
61/61 - 0s - loss: 156134096896.0000 - mean_absolute_pe

Epoch 178/1500
61/61 - 0s - loss: 150542172160.0000 - mean_absolute_percentage_error: 27.3481
Epoch 179/1500
61/61 - 0s - loss: 150559981568.0000 - mean_absolute_percentage_error: 27.2395
Epoch 180/1500
61/61 - 0s - loss: 150435856384.0000 - mean_absolute_percentage_error: 27.2401
Epoch 181/1500
61/61 - 0s - loss: 150302474240.0000 - mean_absolute_percentage_error: 27.2405
Epoch 182/1500
61/61 - 0s - loss: 150739255296.0000 - mean_absolute_percentage_error: 27.2016
Epoch 183/1500
61/61 - 0s - loss: 150034415616.0000 - mean_absolute_percentage_error: 27.2561
Epoch 184/1500
61/61 - 0s - loss: 150133391360.0000 - mean_absolute_percentage_error: 27.1754
Epoch 185/1500
61/61 - 0s - loss: 150032171008.0000 - mean_absolute_percentage_error: 27.0357
Epoch 186/1500
61/61 - 0s - loss: 149817327616.0000 - mean_absolute_percentage_error: 27.1974
Epoch 187/1500
61/61 - 0s - loss: 149760688128.0000 - mean_absolute_percentage_error: 27.1245
Epoch 188/1500
61/61 - 0s - loss: 149585936384.0000 - mean_a

61/61 - 0s - loss: 127668436992.0000 - mean_absolute_percentage_error: 22.1292
Epoch 266/1500
61/61 - 0s - loss: 128791764992.0000 - mean_absolute_percentage_error: 22.5580
Epoch 267/1500
61/61 - 0s - loss: 129380974592.0000 - mean_absolute_percentage_error: 22.7261
Epoch 268/1500
61/61 - 0s - loss: 128209920000.0000 - mean_absolute_percentage_error: 22.4760
Epoch 269/1500
61/61 - 0s - loss: 127565840384.0000 - mean_absolute_percentage_error: 22.5124
Epoch 270/1500
61/61 - 0s - loss: 128535207936.0000 - mean_absolute_percentage_error: 22.4118
Epoch 271/1500
61/61 - 0s - loss: 128826720256.0000 - mean_absolute_percentage_error: 22.4247
Epoch 272/1500
61/61 - 0s - loss: 127664988160.0000 - mean_absolute_percentage_error: 22.3550
Epoch 273/1500
61/61 - 0s - loss: 129315479552.0000 - mean_absolute_percentage_error: 22.6340
Epoch 274/1500
61/61 - 0s - loss: 126647926784.0000 - mean_absolute_percentage_error: 22.2098
Epoch 275/1500
61/61 - 0s - loss: 128802127872.0000 - mean_absolute_percent

Epoch 353/1500
61/61 - 0s - loss: 124338339840.0000 - mean_absolute_percentage_error: 21.9336
Epoch 354/1500
61/61 - 0s - loss: 136706220032.0000 - mean_absolute_percentage_error: 23.9498
Epoch 355/1500
61/61 - 0s - loss: 126333034496.0000 - mean_absolute_percentage_error: 22.7503
Epoch 356/1500
61/61 - 0s - loss: 125646692352.0000 - mean_absolute_percentage_error: 22.1189
Epoch 357/1500
61/61 - 0s - loss: 125688209408.0000 - mean_absolute_percentage_error: 22.2189
Epoch 358/1500
61/61 - 0s - loss: 124854501376.0000 - mean_absolute_percentage_error: 22.2644
Epoch 359/1500
61/61 - 0s - loss: 123864219648.0000 - mean_absolute_percentage_error: 21.9381
Epoch 360/1500
61/61 - 0s - loss: 126541357056.0000 - mean_absolute_percentage_error: 22.8091
Epoch 361/1500
61/61 - 0s - loss: 126217150464.0000 - mean_absolute_percentage_error: 22.0380
Epoch 362/1500
61/61 - 0s - loss: 128463740928.0000 - mean_absolute_percentage_error: 22.7373
Epoch 363/1500
61/61 - 0s - loss: 123042553856.0000 - mean_a

...

Epoch 1139/1500
61/61 - 0s - loss: 115448029184.0000 - mean_absolute_percentage_error: 21.3314
Epoch 1140/1500
61/61 - 0s - loss: 111941861376.0000 - mean_absolute_percentage_error: 22.1118
Epoch 1141/1500
61/61 - 0s - loss: 113024081920.0000 - mean_absolute_percentage_error: 21.1269
Epoch 1142/1500
61/61 - 0s - loss: 115038527488.0000 - mean_absolute_percentage_error: 21.1175
Epoch 1143/1500
61/61 - 0s - loss: 103320748032.0000 - mean_absolute_percentage_error: 20.1981
Epoch 1144/1500
61/61 - 0s - loss: 123514249216.0000 - mean_absolute_percentage_error: 22.7518
Epoch 1145/1500
61/61 - 0s - loss: 126875049984.0000 - mean_absolute_percentage_error: 21.6505
Epoch 1146/1500
61/61 - 0s - loss: 147523436544.0000 - mean_absolute_percentage_error: 25.7878
Epoch 1147/1500
61/61 - 0s - loss: 137665363968.0000 - mean_absolute_percentage_error: 23.9025
Epoch 1148/1500
61/61 - 0s - loss: 119765286912.0000 - mean_absolute_percentage_error: 23.0217
Epoch 1149/1500
61/61 - 0s - loss: 111550578688.00

Epoch 1226/1500
61/61 - 0s - loss: 112062390272.0000 - mean_absolute_percentage_error: 20.9320
Epoch 1227/1500
61/61 - 0s - loss: 117069381632.0000 - mean_absolute_percentage_error: 21.9498
Epoch 1228/1500
61/61 - 0s - loss: 106491428864.0000 - mean_absolute_percentage_error: 20.1801
Epoch 1229/1500
61/61 - 0s - loss: 115751133184.0000 - mean_absolute_percentage_error: 21.8072
Epoch 1230/1500
61/61 - 0s - loss: 115151749120.0000 - mean_absolute_percentage_error: 22.0523
Epoch 1231/1500
61/61 - 0s - loss: 126846582784.0000 - mean_absolute_percentage_error: 23.1200
Epoch 1232/1500
61/61 - 0s - loss: 105855885312.0000 - mean_absolute_percentage_error: 20.7491
Epoch 1233/1500
61/61 - 0s - loss: 113279369216.0000 - mean_absolute_percentage_error: 21.4578
Epoch 1234/1500
61/61 - 0s - loss: 113715101696.0000 - mean_absolute_percentage_error: 21.8608
Epoch 1235/1500
61/61 - 0s - loss: 113594376192.0000 - mean_absolute_percentage_error: 21.5673
Epoch 1236/1500
61/61 - 0s - loss: 116971626496.00

Epoch 1313/1500
61/61 - 0s - loss: 118962470912.0000 - mean_absolute_percentage_error: 21.7695
Epoch 1314/1500
61/61 - 0s - loss: 148914307072.0000 - mean_absolute_percentage_error: 25.0680
Epoch 1315/1500
61/61 - 0s - loss: 122484318208.0000 - mean_absolute_percentage_error: 22.0159
Epoch 1316/1500
61/61 - 0s - loss: 114724552704.0000 - mean_absolute_percentage_error: 20.9700
Epoch 1317/1500
61/61 - 0s - loss: 122060349440.0000 - mean_absolute_percentage_error: 22.7067
Epoch 1318/1500
61/61 - 0s - loss: 116188667904.0000 - mean_absolute_percentage_error: 21.4731
Epoch 1319/1500
61/61 - 0s - loss: 114080366592.0000 - mean_absolute_percentage_error: 21.0521
Epoch 1320/1500
61/61 - 0s - loss: 106447675392.0000 - mean_absolute_percentage_error: 20.5765
Epoch 1321/1500
61/61 - 0s - loss: 111347335168.0000 - mean_absolute_percentage_error: 20.7870
Epoch 1322/1500
61/61 - 0s - loss: 101378760704.0000 - mean_absolute_percentage_error: 19.9506
Epoch 1323/1500
61/61 - 0s - loss: 110501036032.00

Epoch 1400/1500
61/61 - 0s - loss: 103947509760.0000 - mean_absolute_percentage_error: 20.1764
Epoch 1401/1500
61/61 - 0s - loss: 130126012416.0000 - mean_absolute_percentage_error: 23.5656
Epoch 1402/1500
61/61 - 0s - loss: 137029033984.0000 - mean_absolute_percentage_error: 24.2984
Epoch 1403/1500
61/61 - 0s - loss: 124364275712.0000 - mean_absolute_percentage_error: 23.1417
Epoch 1404/1500
61/61 - 0s - loss: 117782241280.0000 - mean_absolute_percentage_error: 21.3889
Epoch 1405/1500
61/61 - 0s - loss: 112638885888.0000 - mean_absolute_percentage_error: 20.9250
Epoch 1406/1500
61/61 - 0s - loss: 105489022976.0000 - mean_absolute_percentage_error: 20.0187
Epoch 1407/1500
61/61 - 0s - loss: 109113507840.0000 - mean_absolute_percentage_error: 21.1889
Epoch 1408/1500
61/61 - 0s - loss: 118902005760.0000 - mean_absolute_percentage_error: 22.3123
Epoch 1409/1500
61/61 - 0s - loss: 119161798656.0000 - mean_absolute_percentage_error: 21.6014
Epoch 1410/1500
61/61 - 0s - loss: 117304139776.00

Epoch 1487/1500
61/61 - 0s - loss: 113629224960.0000 - mean_absolute_percentage_error: 21.0575
Epoch 1488/1500
61/61 - 0s - loss: 114897158144.0000 - mean_absolute_percentage_error: 21.1598
Epoch 1489/1500
61/61 - 0s - loss: 118039035904.0000 - mean_absolute_percentage_error: 21.5042
Epoch 1490/1500
61/61 - 0s - loss: 117029380096.0000 - mean_absolute_percentage_error: 21.4174
Epoch 1491/1500
61/61 - 0s - loss: 117045829632.0000 - mean_absolute_percentage_error: 21.4584
Epoch 1492/1500
61/61 - 0s - loss: 108578439168.0000 - mean_absolute_percentage_error: 20.3889
Epoch 1493/1500
61/61 - 0s - loss: 113503690752.0000 - mean_absolute_percentage_error: 21.1040
Epoch 1494/1500
61/61 - 0s - loss: 116271333376.0000 - mean_absolute_percentage_error: 21.5280
Epoch 1495/1500
61/61 - 0s - loss: 111046598656.0000 - mean_absolute_percentage_error: 20.8612
Epoch 1496/1500
61/61 - 0s - loss: 109574627328.0000 - mean_absolute_percentage_error: 20.4825
Epoch 1497/1500
61/61 - 0s - loss: 111747604480.00